# Preparation

In [ ]:
!pip install cclib
!pip install py3Dmol

import cclib
import os
import py3Dmol
import matplotlib.colors as mcolors
from cclib.parser.utils import PeriodicTable
import numpy as np

## Extract structures

In [ ]:
# Parse the Gaussian output file
log_file_path = 'path/to/log/file/gaussian.log'
data = cclib.io.ccread(log_file_path)

# Get the final optimized coordinates
final_coords = data.atomcoords[-1]

# Get the atomic numbers
atomic_numbers = data.atomnos

# Get the atomic symbols
atomic_symbols = [cclib.parser.utils.PeriodicTable().element[Z] for Z in atomic_numbers]

# Print the coordinates in XYZ format
print(len(atomic_numbers))
print("Final optimized geometry")
for symbol, coords in zip(atomic_symbols, final_coords):
    print(f"{symbol} {coords[0]:.6f} {coords[1]:.6f} {coords[2]:.6f}")

# Create XYZ string
xyz = f"{len(atomic_symbols)}\nOptimized structure\n"
for symbol, coords in zip(atomic_symbols, final_coords):
    xyz += f"{symbol} {coords[0]:.6f} {coords[1]:.6f} {coords[2]:.6f}\n"

## Visualize the Molecule

In [ ]:
# Create a py3Dmol view
view = py3Dmol.view(width=800, height=400)
view.addModel(xyz, "xyz")
view.setStyle({'stick': {}})
view.zoomTo()
view.setStyle({'sphere':{'radius':0.3}, 'stick':{'radius':0.2}})

# Show the view
view.show()

### Extract relevent data

In [ ]:
def extract_gaussian_data(log_file_path):
    # Check if the file exists
    if not os.path.exists(log_file_path):
        raise FileNotFoundError(f"The file {log_file_path} does not exist.")

    # Parse the Gaussian log file
    parser = cclib.io.ccopen(log_file_path)
    data = parser.parse()

    # Create a dictionary to store all the extracted information
    gaussian_data = {}

    # Extract general information
    try:
        gaussian_data['number_of_atoms'] = data.natom
        gaussian_data['atom_numbers'] = data.atomnos
        gaussian_data['atom_coordinates'] = data.atomcoords
        gaussian_data['charge'] = data.charge
        gaussian_data['multiplicity'] = data.mult
    except AttributeError:
        pass

    # Extract SCF and energy-related information
    try:
        gaussian_data['scf_energies'] = data.scfenergies
        gaussian_data['final_energy'] = data.scfenergies[-1] if len(data.scfenergies) > 0 else None
    except AttributeError:
        pass

    try:
        gaussian_data['mpenergies'] = data.mpenergies
    except AttributeError:
        pass

    try:
        gaussian_data['ccenergies'] = data.ccenergies
    except AttributeError:
        pass

    # Extract convergence and optimization information
    try:
        gaussian_data['geometric_optimization'] = data.optstatus
    except AttributeError:
        pass

    # Extract vibrational frequencies and thermochemistry
    try:
        gaussian_data['vibrational_frequencies'] = data.vibfreqs
        gaussian_data['vibrational_intensities'] = data.vibirs
    except AttributeError:
        pass

    # Extract molecular orbital information
    try:
        gaussian_data['mo_energies'] = data.moenergies
        if len(data.moenergies) > 0 and isinstance(data.homos, int) and len(data.moenergies[0]) > data.homos + 1:
            gaussian_data['homo_lumo_gap'] = (np.max(data.moenergies[0][:data.homos + 1]) -
                                              np.min(data.moenergies[0][data.homos + 1:]))
        else:
            gaussian_data['homo_lumo_gap'] = None
        gaussian_data['number_of_homos'] = data.homos
        gaussian_data['mocoefficients'] = data.mocoeffs
    except AttributeError:
        pass

    # Extract basis set information
    try:
        gaussian_data['gbasis'] = data.gbasis
    except AttributeError:
        pass

    # Extract dipole moments and other properties
    try:
        gaussian_data['dipole_moment'] = data.dipolemoment
    except AttributeError:
        pass

    try:
        gaussian_data['mulliken_charges'] = data.atomcharges.get('mulliken')
    except AttributeError:
        pass

    try:
        gaussian_data['nbo_charges'] = data.atomcharges.get('nbo')
    except AttributeError:
        pass


    # Print all the extracted data
    #for key, value in gaussian_data.items():
    #    print(f"{key}: {value}\n")

    return gaussian_data

# Usage
try:
    gaussian_data = extract_gaussian_data(log_file_path)
except Exception as e:
    print(f"Error: {str(e)}")

    # Convert the dictionary to a pandas DataFrame
gaussian_df = pd.DataFrame({key: [value] if not isinstance(value, list) else value for key, value in gaussian_data.items()})

### All QM informations

In [ ]:
gaussian_df

## Plot SCF Energies

In [ ]:
plt.plot(gaussian_df['scf_energies'].iloc[0], marker='o')
plt.xlabel('SCF Cycle')
plt.ylabel('SCF Energy (a.u.)')
plt.title('SCF Energies Convergence')
plt.grid()
plt.show()

## Visualize Mulliken Charges

In [ ]:
def visualize_molecular_structure(log_file_path):
    # Check if the file exists
    if not os.path.exists(log_file_path):
        raise FileNotFoundError(f"The file {log_file_path} does not exist.")

    # Parse the Gaussian log file
    parser = cclib.io.ccopen(log_file_path)
    data = parser.parse()

    # Extract atomic coordinates, atomic numbers, and Mulliken charges
    try:
        atom_coordinates = data.atomcoords[-1]  # Use the final geometry
        atom_numbers = data.atomnos  # Atomic numbers
        mulliken_charges = data.atomcharges.get('mulliken')  # Mulliken charges
    except AttributeError:
        raise ValueError("The log file does not contain the required information (atomic coordinates, atomic numbers, or Mulliken charges).")

    if mulliken_charges is None:
        raise ValueError("Mulliken charges not found in the Gaussian log file.")

    # Normalize Mulliken charges for color mapping (red for negative, blue for positive)
    norm = mcolors.Normalize(vmin=min(mulliken_charges), vmax=max(mulliken_charges))
    cmap = cm.get_cmap('bwr')  # Red to blue color map
    color_values = [cmap(norm(charge)) for charge in mulliken_charges]
    color_values = ['#{:02x}{:02x}{:02x}'.format(int(r * 255), int(g * 255), int(b * 255)) for r, g, b, _ in color_values]

    # Create XYZ format string from extracted data
    pt = PeriodicTable()
    xyz_string = f"{len(atom_numbers)}\nMolecular Structure\n"
    for atom_number, (x, y, z) in zip(atom_numbers, atom_coordinates):
        xyz_string += f"{pt.element[atom_number]} {x:.4f} {y:.4f} {z:.4f}\n"

    # Set up 3Dmol.js viewer using py3Dmol
    view = py3Dmol.view(width=800, height=600)
    view.addModel(xyz_string, "xyz")
    for i, color in enumerate(color_values):
        view.setStyle({'serial': i + 1}, {'sphere': {'color': color, 'radius': 0.5}, 'stick': {}})
    view.zoomTo()
    view.show()


try:
    visualize_molecular_structure(log_file_path)
except Exception as e:
    print(f"Error: {str(e)}")